In [1]:
from datasets import load_dataset

detox_dataset = load_dataset('s-nlp/paradetox')
detox_dataset

C:\Users\zhuld\anaconda3\envs\pytorch_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 19744
    })
})

In [2]:
import torch 

device = torch.device('cuda') 
device

device(type='cuda')

In [3]:
detox_dataset['train']

Dataset({
    features: ['en_toxic_comment', 'en_neutral_comment'],
    num_rows: 19744
})

In [4]:
from sklearn.model_selection import train_test_split
from datasets import Dataset 

train_val, test = train_test_split(detox_dataset['train'], test_size=0.2, random_state=42) # 20% data goes to test set, random_state for reproducibility
train_val_ds = Dataset.from_dict(train_val)

train, val = train_test_split(train_val_ds, test_size=0.125, random_state=42) # 0.125 * 0.8 = 0.1, which means 10% of the data goes to val set

In [5]:
print('length of train dataset: ', len(train['en_toxic_comment']), len(train['en_toxic_comment']))
print('length of validation dataset: ', len(val['en_toxic_comment']), len(val['en_toxic_comment']))
print('length of test dataset: ', len(test['en_toxic_comment']), len(test['en_toxic_comment']))

length of train dataset:  13820 13820
length of validation dataset:  1975 1975
length of test dataset:  3949 3949


In [6]:
test['en_toxic_comment'][0], test['en_neutral_comment'][0]

('anonymous is so fucking annoying .', 'anonymous is so annoying .')

In [7]:
train_dataset = Dataset.from_dict(train)
val_dataset = Dataset.from_dict(val)
test_dataset = Dataset.from_dict(test)

In [8]:
from datasets import DatasetDict

detox_dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

detox_dataset

DatasetDict({
    train: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 13820
    })
    validation: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 1975
    })
    test: Dataset({
        features: ['en_toxic_comment', 'en_neutral_comment'],
        num_rows: 3949
    })
})

In [9]:
detox_dataset['train'][0], detox_dataset['train'][1]

({'en_toxic_comment': 'will do haha , need to pay for the fuckin thing first haha',
  'en_neutral_comment': 'will do haha , need to pay for the thing first haha'},
 {'en_toxic_comment': 'that dp fresh as shit tho',
  'en_neutral_comment': 'That do fresh as tho'})

In [77]:
import numpy as np 
import pandas as pd
import random
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments
from torch.utils.data import Dataset
import torch
from box import Box

import warnings
warnings.filterwarnings("ignore")

In [115]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [76]:
model 

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [12]:
splits = ['train', 'validation', 'test']

max_len_toxic, av_len_toxic = 0, 0
max_len_neutral, av_len_neutral = 0, 0

for split in splits:
    for item in detox_dataset[split]:
        tox_len = len(item['en_toxic_comment'])
        neu_len = len(item['en_neutral_comment'])
        
        if tox_len > max_len_toxic: 
            max_len_toxic = tox_len
        if neu_len > max_len_neutral:
            max_len_neutral = neu_len

        av_len_toxic += tox_len
        av_len_neutral += neu_len

total = len(detox_dataset['train']) + len(detox_dataset['validation']) + len(detox_dataset['test'])

av_len_toxic /= total 
av_len_neutral /= total 

print('average length of toxic sentences: ', av_len_toxic)
print('maximum length of toxic sentences: ', max_len_toxic)

print('average length of neutral sentences: ', av_len_neutral)
print('maximum length of neutral sentences: ', max_len_neutral)

average length of toxic sentences:  54.838279983792546
maximum length of toxic sentences:  135
average length of neutral sentences:  46.2729943273906
maximum length of neutral sentences:  149


In [13]:
def preprocess_function(inputs, max_length=max_len_neutral):
    model_inputs = tokenizer(inputs['en_toxic_comment'], max_length=max_length, truncation=True)
    labels = tokenizer(inputs['en_neutral_comment'], max_length=max_length, truncation=True)
    model_inputs['labels'] = labels['input_ids']

    return model_inputs 

In [109]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
tokenized_datasets = detox_dataset.map(preprocess_function, batched=True)

Map: 100%|████████████████████████████████████████████████████████████████| 3949/3949 [00:00<00:00, 4087.07 examples/s]


In [27]:
from paradetox.evaluation_detox.metric_tools.style_transfer_accuracy import classify_preds
import numpy as np

In [45]:
args = Box({
    'batch_size': 64,
    'cola_classifier_path': '/content/drive/MyDrive/style_transfer/cola_classifier',
    'wieting_tokenizer_path': 'sim.sp.30k.model',
    'wieting_model_path': 'sim.pt',
    't1': 75., # this is default value
    't2': 70., # this is default value
    't3': 12. # this is default value
})

In [35]:
preds = ['Fuck off']
accuracy_by_sent = classify_preds(args, preds)
accuracy = np.mean(accuracy_by_sent)
print('\n')
print(accuracy)

Calculating style of predictions


Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 49.99it/s]



0.0


In [93]:
import tqdm
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import math

def calc_gpt_ppl(preds, aggregate=True):
    detokenize = lambda x: x.replace(" .", ".").replace(" ,", ",").replace(" !", "!").replace(" ?", "?").replace(" )",
                                                                                                                 ")").replace(
        "( ", "(")

    print('Calculating token-level perplexity')
    gpt_ppl = []

    gpt_model = GPT2LMHeadModel.from_pretrained('gpt2-medium').cuda()
    gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
    gpt_model.eval()

    with torch.no_grad():
        for sent in tqdm.tqdm(preds):
            sent = detokenize(sent)
            if len(sent) == 1:
                sent = sent + '.'
            input_ids = gpt_tokenizer.encode(sent)
            inp = torch.tensor(input_ids).unsqueeze(0).cuda()

            try:
                result = gpt_model(inp, labels=inp, return_dict=True)
                loss = result.loss.item()
            except Exception as e:
                print(f'Got exception "{e}" when calculating gpt perplexity for sentence "{sent}" ({input_ids})')
                loss = 100

            gpt_ppl.append(100 if np.isnan(loss) else math.exp(loss))

    if aggregate:
        return np.mean(gpt_ppl)
    return np.array(gpt_ppl)

In [94]:
calc_gpt_ppl(['what you want?'])

Calculating token-level perplexity


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 55.54it/s]


31.25307249033414

In [100]:
import rouge

evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                        max_n=2,
                        limit_length=True,
                        length_limit=100,
                        length_limit_type='words',
                        apply_avg=True,
                        apply_best=False,
                        alpha=0.5, 
                        weight_factor=1.2,
                        stemming=True)

scores = evaluator.get_scores(['You are idiot'], ['you are stupid'])
scores

{'rouge-1': {'f': 0.6666666666666666,
  'p': 0.6666666666666666,
  'r': 0.6666666666666666},
 'rouge-2': {'f': 0.5, 'p': 0.5, 'r': 0.5},
 'rouge-l': {'f': 0.7132754626224419,
  'p': 0.7132754626224419,
  'r': 0.7132754626224419},
 'rouge-w': {'f': 0.5937190915495906,
  'p': 0.6666666666666666,
  'r': 0.5351610411734872}}

In [59]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zhuld\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [128]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated sentences into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference sentences into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    #decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    #decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    #print(decoded_preds)
    accuracy_by_sent = classify_preds(args, decoded_preds)
    accuracy = np.mean(accuracy_by_sent)
    fluency = calc_gpt_ppl(decoded_preds)
    rouge_scores = evaluator.get_scores(decoded_preds, decoded_labels)
    rouge_scores_dict = {k: round(v['f'], 4) for k, v in rouge_scores.items()}

    results = {'STA': accuracy, 'Fluency': fluency}
    results.update(rouge_scores_dict)
    return results

In [117]:
batch_size = 128
num_train_epochs = 10
logging_steps = len(tokenized_datasets["train"]) // batch_size

arguments = Seq2SeqTrainingArguments(
    output_dir=f"asqiiBART-detox",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    num_train_epochs=num_train_epochs,
    logging_steps=logging_steps,
    use_cpu=False,
    predict_with_generate=True,
    )

In [118]:
trainer = Seq2SeqTrainer(
    model,
    arguments,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,)

In [119]:
trainer.train()

Epoch,Training Loss,Validation Loss,Sta,Fluency,Rouge-1,Rouge-2,Rouge-l,Rouge-w
1,1.197500,0.970657,0.826835,411.665532,0.802300,0.683600,0.822600,0.604900
2,0.950200,0.954055,0.852152,419.786295,0.803900,0.684000,0.824000,0.606000
3,0.853000,0.941986,0.892658,415.124272,0.805500,0.686600,0.825000,0.606600
4,0.784600,0.947858,0.904304,454.407348,0.807500,0.689200,0.826900,0.607800
5,0.728200,0.948358,0.900759,505.193413,0.808400,0.690600,0.827800,0.608800
6,0.687900,0.960254,0.905316,431.799386,0.805800,0.687600,0.825400,0.606800
7,0.646200,0.967517,0.908861,419.147536,0.805000,0.685900,0.824800,0.606300
8,0.620900,0.982985,0.910886,414.437049,0.804800,0.686000,0.824500,0.606100
9,0.601700,0.986766,0.911392,424.757781,0.804700,0.685800,0.824400,0.605900
10,0.583400,0.991538,0.913924,423.944209,0.802900,0.683900,0.822900,0.604500


Calculating style of predictions


Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:21<00:00,  1.43it/s]


Calculating token-level perplexity


100%|██████████████████████████████████████████████████████████████████████████████| 1975/1975 [00:34<00:00, 57.51it/s]


...

Calculating style of predictions


Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:21<00:00,  1.43it/s]


Calculating token-level perplexity


100%|██████████████████████████████████████████████████████████████████████████████| 1975/1975 [00:34<00:00, 57.45it/s]


TrainOutput(global_step=1080, training_loss=0.763519659307268, metrics={'train_runtime': 1527.5427, 'train_samples_per_second': 90.472, 'train_steps_per_second': 0.707, 'total_flos': 2211178302627840.0, 'train_loss': 0.763519659307268, 'epoch': 10.0})

In [120]:
def detoxify(text, max_length=150):
    inputs = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    inputs = inputs.to(device)
    inputs_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']
    output_ids = model.generate(inputs_ids, attention_mask=attention_mask, max_length=150, num_beams=30, early_stopping=False)
    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return output

In [122]:
toxics, trues, preds = [], [], []

for i in range(len(detox_dataset['test'])):
    toxic = detox_dataset['test'][i]['en_toxic_comment']
    neutral_true = detox_dataset['test'][i]['en_neutral_comment']
    neutral_pred = detoxify(toxic)

    toxics.append(toxic)
    trues.append(neutral_true)
    preds.append(neutral_pred)

In [123]:
import pandas as pd 

df = pd.DataFrame({'Toxic': toxics, 'TrueNeutral': trues, 'PredictionNeutral': preds})
df

,Toxic,TrueNeutral,PredictionNeutral
0,anonymous is so fucking annoying .,anonymous is so annoying .,anonymous is so annoying.
1,why would you fucking do anything else ?,Why would you do anything else?,why would you do anything else?
2,"but it doess look good , just fucking shit sca...","but it doess look good , just scarringly good","but it doess look good, just scarringly good"
3,you really been on that wrestling shit again h...,"You've really been into wrestling again, huh?",you really been on that wrestling again huh? haha
4,can we atleast listen to gospel if its gonna b...,can we at least listen to gospel if its gonna ...,can we atleast listen to gospel if its gonna b...
...,...,...,...
3944,i would show u who the bitch is,i would show u who is this,i would show u who she is
3945,i could make jokes but some one would bitch at...,i could make jokes but some one would think wr...,I could make jokes but some one would make fun...
3946,what will it take to fix this shit ?,What will take to fix this problem,What will it take to fix this?
3947,to say the establishment is fighting tooth and...,to say the establishment is fighting tooth and...,to say the establishment is fighting tooth and...


In [124]:
df.to_csv('asqiiBase_predictions.csv', index=False)

### Training BART-Large

In [134]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

In [135]:
def preprocess_function(inputs, max_length=max_len_neutral):
    model_inputs = tokenizer(inputs['en_toxic_comment'], max_length=max_length, truncation=True)
    labels = tokenizer(inputs['en_neutral_comment'], max_length=max_length, truncation=True)
    model_inputs['labels'] = labels['input_ids']

    return model_inputs 

In [136]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
tokenized_datasets = detox_dataset.map(preprocess_function, batched=True)

Map: 100%|████████████████████████████████████████████████████████████████| 3949/3949 [00:00<00:00, 4026.53 examples/s]


In [137]:
batch_size = 128
num_train_epochs = 10
logging_steps = len(tokenized_datasets["train"]) // batch_size

arguments = Seq2SeqTrainingArguments(
    output_dir=f"asqiiBARTLarge-detox",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    num_train_epochs=num_train_epochs,
    logging_steps=logging_steps,
    use_cpu=False,
    predict_with_generate=True,
    )

In [138]:
trainer = Seq2SeqTrainer(
    model,
    arguments,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,)

In [139]:
trainer.train()

Epoch,Training Loss,Validation Loss,Sta,Fluency,Rouge-1,Rouge-2,Rouge-l,Rouge-w
1,1.073200,1.015069,0.869367,387.200583,0.801800,0.683100,0.822100,0.605000
2,0.838600,1.030199,0.904304,369.716240,0.806400,0.689200,0.826400,0.608200
3,0.733100,1.019994,0.924557,752.815062,0.801500,0.682100,0.821600,0.603800
4,0.649700,1.063380,0.925570,452.965838,0.804600,0.685600,0.824600,0.606100
5,0.579200,1.078866,0.927595,494.843183,0.803400,0.682400,0.823700,0.604900
6,0.523300,1.086039,0.923544,335.290332,0.801000,0.679600,0.821300,0.602400
7,0.478900,1.121182,0.936709,340.390177,0.797300,0.672600,0.817500,0.598900
8,0.434100,1.153589,0.930633,444.180831,0.796000,0.672000,0.816600,0.598300
9,0.411600,1.168280,0.940253,329.385371,0.794200,0.669300,0.814900,0.597200
10,0.386700,1.206725,0.930633,345.636060,0.795200,0.670500,0.815400,0.597600


Calculating style of predictions


Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:21<00:00,  1.45it/s]


Calculating token-level perplexity


100%|██████████████████████████████████████████████████████████████████████████████| 1975/1975 [00:34<00:00, 57.66it/s]


...

Calculating style of predictions


Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:21<00:00,  1.45it/s]


Calculating token-level perplexity


100%|██████████████████████████████████████████████████████████████████████████████| 1975/1975 [00:34<00:00, 57.48it/s]


TrainOutput(global_step=1080, training_loss=0.608669744376783, metrics={'train_runtime': 2352.326, 'train_samples_per_second': 58.75, 'train_steps_per_second': 0.459, 'total_flos': 7856406007087104.0, 'train_loss': 0.608669744376783, 'epoch': 10.0})

In [140]:
def detoxify(text, max_length=150):
    inputs = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    inputs = inputs.to(device)
    inputs_ids, attention_mask = inputs['input_ids'], inputs['attention_mask']
    output_ids = model.generate(inputs_ids, attention_mask=attention_mask, max_length=150, num_beams=30, early_stopping=False)
    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return output

In [141]:
toxics, trues, preds = [], [], []

for i in range(len(detox_dataset['test'])):
    toxic = detox_dataset['test'][i]['en_toxic_comment']
    neutral_true = detox_dataset['test'][i]['en_neutral_comment']
    neutral_pred = detoxify(toxic)

    toxics.append(toxic)
    trues.append(neutral_true)
    preds.append(neutral_pred)

In [142]:
import pandas as pd 

df = pd.DataFrame({'Toxic': toxics, 'TrueNeutral': trues, 'PredictionNeutral': preds})
df

,Toxic,TrueNeutral,PredictionNeutral
0,anonymous is so fucking annoying .,anonymous is so annoying .,anonymous is so annoying.
1,why would you fucking do anything else ?,Why would you do anything else?,Why would you do anything else?
2,"but it doess look good , just fucking shit sca...","but it doess look good , just scarringly good","but it doess look good, just scarringly good"
3,you really been on that wrestling shit again h...,"You've really been into wrestling again, huh?",you really been on that wrestling again huh? haha
4,can we atleast listen to gospel if its gonna b...,can we at least listen to gospel if its gonna ...,can we atleast listen to gospel if its gonna b...
...,...,...,...
3944,i would show u who the bitch is,i would show u who is this,i would show u who she is
3945,i could make jokes but some one would bitch at...,i could make jokes but some one would think wr...,i could make jokes but some one would yell at me.
3946,what will it take to fix this shit ?,What will take to fix this problem,What will it take to fix this?
3947,to say the establishment is fighting tooth and...,to say the establishment is fighting tooth and...,to say the establishment is fighting tooth and...


In [143]:
df.to_csv('asqiiLarge_predictions.csv', index=False)

In [155]:
detoxify('This idiot should work at the cafe')

'This person should work at the cafe'

In [156]:
detoxify('This bitch should work at the cafe')

'She should work at the cafe'

In [157]:
detoxify('I wanna kill yall morons')

'I wanna kill all of you'

In [158]:
detoxify('howdy fellas, whats good')

'howdy fellas, whats good'

In [165]:
detoxify("You are an idoit"), detoxify('You are an iddoijd')

('You are not a good person.', 'you are an iddoijd')

In [166]:
detoxify('I want to take a shit now')

'I want to take a drink now'

In [167]:
detoxify('I want to smack your faces')

'I want to hit you'

In [169]:
detoxify('tchknt, haha'), detoxify('tchknt')

('Hahaha.', "I don't care.")